# $\nabla v_{LSR}$ in HC$_3$N

We calculate the gradient in velocities $v_{LSR}$ adn then visualize it with line integral convolution (LIC). 

For this, we calculate the gradient in a plane around a radius of approx 2 beam sizes, to sample well resolved structures.

In [1]:
import numpy as np
import scipy.linalg # we only fit a linear plane so we go with easy linear code
from astropy.io import fits
import astropy.units as u
import sys
sys.path.append('../')
from B5setup import *
import matplotlib.pyplot as plt
from licpy.lic import runlic
%matplotlib inline

2022-11-10 16:45:36.817308: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 16:45:39.409011: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 16:45:39.409050: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-10 16:45:39.545656: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 16:45:43.850092: W tensorflow/stream_executor/platform/de

In [2]:
gaussfitfolder = 'gaussfit/'
velfile = gaussfitfolder + 'B5-NOEMA+30m-H3CN-10-9_cut_K_1G_fitparams_filtered_Vlsr'
nablavelfile = velfile + '_gradient'
epsilon = 1e-3 # tolerance to mask values

In [3]:
# test: data that has to be given as input

mean = np.array([0.0,0.0,0.0])
cov = np.array([[1.0,-0.5,0.8], [-0.5,1.1,0.0], [0.8,0.0,1.0]])
data = np.random.multivariate_normal(mean, cov, 50) # is (number of points, [x, y, z])

# regular grid covering the domain of the data
X,Y = np.meshgrid(np.arange(-3.0, 3.0, 0.5), np.arange(-3.0, 3.0, 0.5))
XX = X.flatten()
YY = Y.flatten()

A = np.c_[data[:,0], data[:,1], np.ones(data.shape[0])] # Design matrix, where we have X, Y and a constant value (we just use 1)
C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])    # coefficients: magnitude in X, magnitude in Y and constant

In [4]:
# we need to sample an area of about 2 beams
velfield, velheader = fits.getdata(velfile+'.fits', header=True)
y_velfield, x_velfield = np.mgrid[:len(velfield), :len(velfield[0])]
beammaj, beammin = velheader['BMAJ'], velheader['BMIN']
pixsize_deg = np.abs(velheader['CDELT2'])
pixsize_pc = (pixsize_deg * 3600 * dist_B5.value * u.au).to(u.pc).value # this is pixel to pc
equivradius = np.sqrt(beammaj * beammin / (4 * np.log(2)))
sampleradius = int(np.round(2* equivradius / pixsize_deg, 0))
beamarea_pix = np.pi * beammaj * beammin / (4 * np.log(2) * pixsize_deg**2)

In [5]:
filled_indices = np.where(~np.isnan(velfield))
nablax_map = np.ones(np.shape(velfield)) * np.nan
nablay_map = np.ones(np.shape(velfield)) * np.nan

In [6]:
for y, x in np.transpose(filled_indices):
    sampleregion = velfield[y-sampleradius:y+sampleradius+1, x-sampleradius:x+sampleradius+1]
    if np.shape(sampleregion)[0] != np.shape(sampleregion)[1]: 
        continue
    elif np.sum(~np.isnan(sampleregion))<beamarea_pix: 
        continue
    X,Y = np.meshgrid(np.arange(-sampleradius, sampleradius+1, 1), np.arange(-sampleradius, sampleradius+1, 1))
    # XX = X.flatten()
    # YY = Y.flatten()
    # sampleradius+1 will be the center
    index_sampleregion_filter = np.where(distancepix(X, Y, sampleradius+1, sampleradius+1)<sampleradius * ~np.isnan(sampleregion))
    data_sample = np.transpose([X[index_sampleregion_filter]*pixsize_pc, Y[index_sampleregion_filter]*pixsize_pc, sampleregion[index_sampleregion_filter]])
    # data_sample_filtered = data_sample[:, ~np.isnan(data_sample[:, 2])]
    A = np.c_[data_sample[:,0],data_sample[:,1], np.ones(data_sample.shape[0])] 
    C,_,_,_ = scipy.linalg.lstsq(A, data_sample[:,2])
    if np.abs(C[0])< epsilon or np.abs(C[1])< epsilon: continue
    nablax_map[y, x] = C[0] 
    nablay_map[y, x] = C[1] 
    
    # we need to get the gradient in km/s/pc

In [7]:
gradheader = velheader.copy()
gradheader['BUNIT'] = 'km s-1 pc-1'

fits.writeto('gradient_x_test.fits', nablax_map, gradheader, overwrite=True)
fits.writeto('gradient_y_test.fits', nablay_map, velheader, overwrite=True)

In [8]:
tex = runlic(nablax_map, nablay_map, 10)


AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [ ]:
plt.imshow(tex)